In [2]:
# Beam vs Greedy Stepwise Regression Evaluation Script

import numpy as np
import pandas as pd
from sklearn.datasets import make_regression
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error

# Function to create correlated synthetic datasets
def make_correlated_data(n_samples=300, n_features=20, n_informative=5, noise=30.0, corr=0.9):
    rng = np.random.RandomState(42)
    X = rng.normal(size=(n_samples, n_features))
    for i in range(n_features):
        if i >= n_informative:
            X[:, i] = X[:, i % n_informative] * corr + rng.normal(scale=0.1, size=n_samples)
    coef = np.zeros(n_features)
    coef[:n_informative] = rng.uniform(1, 10, size=n_informative)
    y = X @ coef + rng.normal(scale=noise, size=n_samples)
    return pd.DataFrame(X, columns=[f"x{i}" for i in range(n_features)]), y

# Beam stepwise regression with MSE tracking
def track_beam_forward_mse(X_train, y_train, X_test, y_test, max_vars=10, beam_width=3):
    all_vars = list(X_train.columns)
    beam = [([], float("inf"))]
    track = []
    for _ in range(max_vars):
        new_beam = []
        for used, _ in beam:
            remaining = [v for v in all_vars if v not in used]
            for v in remaining:
                vars_now = used + [v]
                reg = LinearRegression().fit(X_train[vars_now], y_train)
                preds = reg.predict(X_test[vars_now])
                mse = mean_squared_error(y_test, preds)
                new_beam.append((vars_now, mse))
        beam = sorted(new_beam, key=lambda x: x[1])[:beam_width]
        track.append((beam[0][0], beam[0][1]))
    return track

# Greedy forward stepwise regression with MSE tracking
def track_greedy_forward_mse(X_train, y_train, X_test, y_test, max_vars=10):
    all_vars = list(X_train.columns)
    selected_vars = []
    mse_track = []
    for _ in range(max_vars):
        best_var = None
        best_mse = float("inf")
        for v in all_vars:
            if v in selected_vars:
                continue
            vars_now = selected_vars + [v]
            reg = LinearRegression().fit(X_train[vars_now], y_train)
            preds = reg.predict(X_test[vars_now])
            mse = mean_squared_error(y_test, preds)
            if mse < best_mse:
                best_mse = mse
                best_var = v
        if best_var is not None:
            selected_vars.append(best_var)
            mse_track.append((selected_vars.copy(), best_mse))
    return mse_track

# Generate multiple hard synthetic datasets
def generate_datasets():
    datasets = {}
    datasets["Synthetic-30F-Corr"] = make_correlated_data(n_features=30, noise=25.0, corr=0.7)
    datasets["Synthetic-HighNoise"] = make_correlated_data(n_features=20, noise=50.0, corr=0.6)
    datasets["Synthetic-LowSNR"] = make_correlated_data(n_features=25, n_informative=3, noise=60.0, corr=0.95)
    return datasets

# Run evaluation and save results
results = []
for name, (X, y) in generate_datasets().items():
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)
    beam_track = track_beam_forward_mse(X_train, y_train, X_test, y_test)
    greedy_track = track_greedy_forward_mse(X_train, y_train, X_test, y_test)
    for i, ((_, beam_mse), (_, greedy_mse)) in enumerate(zip(beam_track, greedy_track)):
        results.append({
            "Dataset": name,
            "Step": i + 1,
            "Beam MSE": beam_mse,
            "Greedy MSE": greedy_mse
        })


pd.DataFrame(results)

,Dataset,Step,Beam MSE,Greedy MSE
0,Synthetic-30F-Corr,1,636.824409,636.824409
1,Synthetic-30F-Corr,2,567.134512,567.134512
2,Synthetic-30F-Corr,3,538.946541,543.190239
3,Synthetic-30F-Corr,4,531.104927,534.733333
4,Synthetic-30F-Corr,5,524.190535,527.424201
5,Synthetic-30F-Corr,6,517.022857,520.921089
6,Synthetic-30F-Corr,7,511.483228,518.613444
7,Synthetic-30F-Corr,8,507.118041,516.772871
8,Synthetic-30F-Corr,9,504.083023,515.019802
9,Synthetic-30F-Corr,10,502.522334,513.705491
